In [ ]:
# import requests

# url = "http://www.camara.leg.br/cotas/Ano-2019.csv.zip"
# r = requests.get(url)

In [ ]:
# file = open("Ano-2019.csv.zip", "wb")
# file.write(r.content)
# file.close()

In [ ]:
# from zipfile import ZipFile

# zip_file = ZipFile("Ano-2019.csv.zip", 'r')

In [ ]:
# zip_file.extract(member="Ano-2019.csv", path="reembolso-2019")

In [ ]:
# zip_file.close()

In [ ]:
import pandas as pd

In [ ]:
DTYPE = {
    'txNomeParlamentar': str,
    'ideCadastro': str,
    'nuCarteiraParlamentar': str,
    'nuLegislatura': str,
    'sgUF': str,
    'sgPartido': str,
    'codLegislatura': str,
    'numSubCota': str,
    'txtDescricao': str,
    'numEspecificacaoSubCota': str,
    'txtDescricaoEspecificacao': str,
    'txtFornecedor': str,
    'txtCNPJCPF': str,
    'txtNumero': str,
    'indTipoDocumento': str,
    'datEmissao': str,
    'vlrDocumento': float,
    'vlrGlosa': str,
    'vlrLiquido': float,
    'numMes': str,
    'numAno': str,
    'numParcela': str,
    'txtPassageiro': str,
    'txtTrecho': str,
    'numLote': str,
    'numRessarcimento': str,
    'nuDeputadoId': str,
    'ideDocumento': str,
}

df_reembolso = pd.read_csv("reembolso-2019/Ano-2019.csv", sep=";", dtype=DTYPE) 

In [ ]:
df_reembolso.head(1)

In [ ]:
df_reembolso.sample(10)

In [ ]:
filtro_partido_null = df_reembolso.cpf.isna()
# filtro_partido_null = df_reembolso['sg-Partido'].isna()

df_reembolso.loc[filtro_partido_null, "cpf"] = "0000000"

In [ ]:
df_reembolso.head()

In [ ]:
# df_reembolso["datEmissao"]

In [ ]:
df_reembolso["datEmissao"] = pd.to_datetime(df_reembolso["datEmissao"], format="%Y-%m-%d") 

In [ ]:
# pegando dados do final de semana
# df_reembolso["datEmissao"].dt.dayofweek < 5

In [ ]:
df_reembolso["txtCNPJCPF"] = df_reembolso["txtCNPJCPF"].str.replace(r'\D', '', regex=True)

In [ ]:
df_reembolso["txtCNPJCPF"]

In [ ]:
# usando a opção de normalizar o resultado do value_counts
# também ordenamos por ordem decrescente
df_reembolso.txtDescricao.value_counts(normalize=True).sort_values(ascending=False).to_frame().head(10)

In [ ]:
df_reembolso["vlrLiquido"].describe()

In [ ]:
# usamos 2 filtros aqui, para txtDescricao e para vlrLiquido

filtro_cat_alim = df_reembolso.txtDescricao == 'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR'
filtro_vlr_max = df_reembolso.vlrLiquido > 5000

df_reembolso[(filtro_cat_alim) & (filtro_vlr_max)]

In [ ]:
# aqui nós agrupamos por partido, UF e nome do parlamentar. 
# Somamos o vlrLiquido considerando os grupos gerados pelo groupby
# ordenamos de forma decrescente 

df_reembolso.groupby(
    ['sgPartido', 'sgUF', 'txNomeParlamentar']
)["vlrLiquido"].sum().sort_values(ascending=False).to_frame()

In [ ]:
# aqui nós configuramos para o pandas mostrar números decimais considerando somente
# 2 casas decimais depois do ponto 
pd.options.display.float_format = '{:,.2f}'.format

# agrupamos os dados por txtCNPJCPF e txtFornecedor. 
# somamos o vlrLiquido considerando o grupo resultante pelo groupby
# nós pegamos tbm só os 10 primeiros itens através do [:10]
df_reembolso.groupby(['txtCNPJCPF', 'txtFornecedor'])['vlrLiquido'].sum().sort_values(
    ascending=False
).to_frame()[:10]

###  Receita Federal

In [ ]:
DTYPE = {
    'cnpj': str
}

df_empresas = pd.read_csv(
    '2019-11-19-companies.csv',
    dtype=DTYPE,  
)

In [ ]:
# usamos essa configuração para o pandas mostrar todas as colunas do dataframe. 
# se preferir, mude ela para o início do notebook, pq essa configuração só vale
# depois que vc executar esse comando
pd.set_option('display.max_columns', None)

In [ ]:
df_empresas.head(1)

In [ ]:
# fizemos o merge da tabela df_reembolso com df_empresas.
# LEMBRANDO: 
# df_reembolso é considerada a tabela da esquerda, então ela é a left.
# df_empresas é considerada a tabela da direita, então ela é a right.
df_final = df_reembolso.merge(
    df_empresas, 
    how='left', 
    left_on='txtCNPJCPF', 
    right_on='cnpj'
)

In [ ]:
df_final[['txtCNPJCPF', 'cnpj']]

In [ ]:
# situation == 8 significa empresa que está BAIXADA
f = df_final.situation == 8 

df_final[f].shape

In [ ]:
# datEmissao é a coluna que informa a data em que o gasto foi feito (vem da base da CEAP)
# situation_date é a coluna que informa a data em que a empresa ficou com a situação da coluna situation (vem da Receita Federal)
df_final["datEmissao"] = pd.to_datetime(df_final["datEmissao"], format="%Y-%m-%d") 
df_final['situation_date'] = pd.to_datetime(df_final["situation_date"], format="%Y-%m-%d") 

Lembrando, se temos `datEmissao` como `02/10/2020` e `situation_date` como `01/10/2020`, o python/pandas vai entender que `datEmissao` contém uma data posterior a `situation_date`, logo, é uma _data maior_. Por isso podemos fazer algo como `datEmissao` > `situation_date`, isso vai ser verdadeiro se a `datEmissao` for uma data que veio depois de `situation_date`.

In [ ]:
# aqui vamos pesquisar se a data do gasto veio depois da data em que a empresa mudou de situação
# vamos agrupar por txNomeParlamentar
# aplicamos count em ideCadastro e sum em vlrLiquido
# ordenamos o resultado por vlrLiquido
(
    df_final[f].query('datEmissao > situation_date')
    .groupby('txNomeParlamentar')
    .agg({'ideCadastro': 'count', 'vlrLiquido': 'sum'})
    .sort_values(by='vlrLiquido', ascending=False)
    .reset_index()
)

In [ ]:
df_final.columns

In [ ]:
(
    df_final[f].query('datEmissao > situation_date')
    .groupby(['txNomeParlamentar', 'txtFornecedor', 'situation'])['ideCadastro'].count()
    .sort_values(ascending=False).to_frame()
)